In [25]:
from neo4j import GraphDatabase
import geopandas as gpd

creds = r"C:\Users\adria\Desktop\STUDIA_FOLDERY\wyznaczanie-trasy\neo4j\credentials.txt"
shp = r"C:\Users\adria\Desktop\STUDIA_FOLDERY\wyznaczanie-trasy\skjz\direction_calosc_0.shp"

with open (creds, "r") as file:
    lines = file.readlines()
    uri = lines[0].strip()
    user = lines[1].strip()
    password = lines[2].strip()

driver = GraphDatabase.driver(uri, auth=(user, password))

In [26]:
road_classes_speed = {
    "A": 140,
    "S": 120,
    "GP": 100,
    "G": 80,
    "Z": 60,
    "L": 50,
    "D": 40,
    "I": 30,
}

for key in road_classes_speed:
    road_classes_speed[key] /= 2

In [27]:
gdf = gpd.read_file(shp)

with driver.session() as session:
    for idx,row in gdf.iterrows():
        length = row.geometry.length
        coords = list(row.geometry.coords)
        startPoint = coords[0]
        endPoint = coords[-1]
        x1 = startPoint[0]
        y1 = startPoint[1]
        x2 = endPoint[0]
        y2 = endPoint[1]
        
        edge_id = idx
        length = row.geometry.length
        road_class_speed = road_classes_speed[row['klasaDrogi']]
        time = length/(road_class_speed * 1000 / 3600)
        direction = row['kierunek']
        
        query = f"""
        MERGE (n:Node {{id: apoc.util.md5([{int(x1)}, {int(y1)}]), location: point({{x: {x1}, y: {y1}}})}})
        MERGE (m:Node {{id: apoc.util.md5([{int(x2)}, {int(y2)}]), location: point({{x: {x2}, y: {y2}}})}})
        """
        if direction == 0:
            query += f"""
            MERGE (n)-[:CONNECTION {{edge_id: {edge_id}, length: {length}, time: {time}}}]->(m)
            MERGE (m)-[:CONNECTION {{edge_id: {edge_id}, length: {length}, time: {time}}}]->(n)
            RETURN n, m;
            """
        session.run(query)

In [41]:
import requests
import json


InvalidSchema: No connection adapters were found for 'neo4j+s://260dfa0b.databases.neo4j.io'